In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

## Modelagem e Normalização dos dados de treino

#### Lendo os dados escolidos para treinar o modelo

In [2]:
df_prospects_prospects = pd.read_csv("dados/df_prospects_prospects.csv")

df_applicants_cv_pt = pd.read_csv("dados/df_applicants_cv_pt.csv")

df_vagas_informacoes_basicas = pd.read_csv("dados/df_vagas_informacoes_basicas.csv")
df_vagas_perfil_vaga = pd.read_csv("dados/df_vagas_perfil_vaga.csv")

#### Removendo as colunas não utilizadas

In [3]:
df_prospects_prospects = df_prospects_prospects[["codigo_condidato", "codigo_vaga", "situacao_candidado"]]
df_vagas_informacoes_basicas = df_vagas_informacoes_basicas[["codigo_vaga", "titulo_vaga", "cliente", "tipo_contratacao"]]
df_vagas_perfil_vaga = df_vagas_perfil_vaga[["codigo_vaga", "vaga_especifica_para_pcd", "nivel profissional", "nivel_academico", "nivel_ingles", "principais_atividades", "competencia_tecnicas_e_comportamentais"]]

#### Criando um dataset unificado

In [4]:
df_unido = pd.merge(df_prospects_prospects, df_vagas_informacoes_basicas, on='codigo_vaga')
df_unido = pd.merge(df_unido, df_vagas_perfil_vaga, on='codigo_vaga')
df_unido = pd.merge(df_unido, df_applicants_cv_pt, on='codigo_condidato')
df_unido.head(5)

,codigo_condidato,codigo_vaga,situacao_candidado,titulo_vaga,cliente,tipo_contratacao,vaga_especifica_para_pcd,nivel profissional,nivel_academico,nivel_ingles,principais_atividades,competencia_tecnicas_e_comportamentais,cv_pt
0,25632,4530,Encaminhado ao Requisitante,CONSULTOR CONTROL M,"Morris, Moran and Dodson",PJ/Autônomo,Não,Pleno,Ensino Superior Completo,Nenhum,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,\ndados pessoais\nestado civil: casado\nidade:...
1,25529,4530,Encaminhado ao Requisitante,CONSULTOR CONTROL M,"Morris, Moran and Dodson",PJ/Autônomo,Não,Pleno,Ensino Superior Completo,Nenhum,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,"solteiro, 47 anos\n\nestrada meringuava, nº 17..."
2,25364,4531,Contratado pela Decision,2021-2607395-PeopleSoft Application Engine-Dom...,Gonzalez and Sons,PJ/Autônomo,Não,Sênior,Ensino Médio Completo,Nenhum,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,\n\nárea de atuação: lider de consultoria / ge...
3,25360,4531,Encaminhado ao Requisitante,2021-2607395-PeopleSoft Application Engine-Dom...,Gonzalez and Sons,PJ/Autônomo,Não,Sênior,Ensino Médio Completo,Nenhum,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,informações pessoais\n estado civil: casado\n...
4,26338,4533,Contratado pela Decision,2021-2605708-Microfocus Application Life Cycle...,Barnes-Woods,PJ/Autônomo,Não,Sênior,Ensino Médio Completo,Técnico,Arquiteto\n\nFoco na área e automação.\n\nRequ...,Arquiteto\n\nFoco na área e automação.\n\nRequ...,"solteiro, brasileiro, 21/06/1987\nhabilitação ..."


## Limpeza dos dados de treino

#### Função para printas as colunas vazias de um DataFrame

In [5]:
def print_null_cell(df: pd.DataFrame):
    print(f"Total de linha: {len(df_unido)}\n")
    print("Quantidade de células vazias:")
    for con in df.columns:
        qtd_vazias = ((df[con] == '') | (df[con].isnull())).sum()
        print(f"{con}: {qtd_vazias}")

In [6]:
print_null_cell(df_unido)

Total de linha: 45071

Quantidade de células vazias:
codigo_condidato: 0
codigo_vaga: 0
situacao_candidado: 0
titulo_vaga: 0
cliente: 0
tipo_contratacao: 1771
vaga_especifica_para_pcd: 4797
nivel profissional: 0
nivel_academico: 0
nivel_ingles: 0
principais_atividades: 24
competencia_tecnicas_e_comportamentais: 26
cv_pt: 2552


In [7]:
# Aqui estou presumindo que todas as vagas que tenham o campo "vaga_especifica_para_pcd" vazio não são especificas para PCD
df_unido['vaga_especifica_para_pcd'] = df_unido['vaga_especifica_para_pcd'].replace('', pd.NA).fillna('Não')
print_null_cell(df_unido)

Total de linha: 45071

Quantidade de células vazias:
codigo_condidato: 0
codigo_vaga: 0
situacao_candidado: 0
titulo_vaga: 0
cliente: 0
tipo_contratacao: 1771
vaga_especifica_para_pcd: 0
nivel profissional: 0
nivel_academico: 0
nivel_ingles: 0
principais_atividades: 24
competencia_tecnicas_e_comportamentais: 26
cv_pt: 2552


In [8]:
# Aqui estou removendo odas as linhas que tenham células vazias 
df_unido = df_unido.dropna()
print_null_cell(df_unido)

Total de linha: 40838

Quantidade de células vazias:
codigo_condidato: 0
codigo_vaga: 0
situacao_candidado: 0
titulo_vaga: 0
cliente: 0
tipo_contratacao: 0
vaga_especifica_para_pcd: 0
nivel profissional: 0
nivel_academico: 0
nivel_ingles: 0
principais_atividades: 0
competencia_tecnicas_e_comportamentais: 0
cv_pt: 0


In [9]:
df_unido["situacao_candidado"].value_counts()

situacao_candidado
Prospect                          15276
Encaminhado ao Requisitante       12271
Inscrito                           3051
Não Aprovado pelo Cliente          2605
Contratado pela Decision           1953
Desistiu                           1852
Não Aprovado pelo RH               1506
Não Aprovado pelo Requisitante      570
Entrevista Técnica                  427
Sem interesse nesta vaga            323
Entrevista com Cliente              319
Em avaliação pelo RH                316
Aprovado                            159
Contratado como Hunting             152
Desistiu da Contratação              46
Documentação PJ                       4
Documentação CLT                      3
Documentação Cooperado                2
Recusado                              1
Encaminhar Proposta                   1
Proposta Aceita                       1
Name: count, dtype: int64

#### criando a coluna de gabarito do modelo

In [10]:
df_unido['situacao_candidado'] = df_unido['situacao_candidado'].replace('Proposta Aceita', 'sim')
df_unido['situacao_candidado'] = df_unido['situacao_candidado'].replace('Encaminhado ao Requisitante', 'sim')
df_unido['situacao_candidado'] = df_unido['situacao_candidado'].replace('Aprovado', 'sim')
df_unido['situacao_candidado'] = df_unido['situacao_candidado'].replace('Não Aprovado pelo Cliente', 'não')
df_unido['situacao_candidado'] = df_unido['situacao_candidado'].replace('Não Aprovado pelo RH', 'não')
df_unido['situacao_candidado'] = df_unido['situacao_candidado'].replace('Não Aprovado pelo Requisitante', 'não')
df_unido["situacao_candidado"].value_counts()

situacao_candidado
Prospect                    15276
sim                         12431
não                          4681
Inscrito                     3051
Contratado pela Decision     1953
Desistiu                     1852
Entrevista Técnica            427
Sem interesse nesta vaga      323
Entrevista com Cliente        319
Em avaliação pelo RH          316
Contratado como Hunting       152
Desistiu da Contratação        46
Documentação PJ                 4
Documentação CLT                3
Documentação Cooperado          2
Recusado                        1
Encaminhar Proposta             1
Name: count, dtype: int64

In [11]:
# removendo todas as linha em que o valor da coluna situacao_candidado é diferente de sim ou não
df_unido = df_unido[df_unido['situacao_candidado'].isin(['sim', 'não'])]
df_unido["situacao_candidado"].value_counts()

situacao_candidado
sim    12431
não     4681
Name: count, dtype: int64

In [12]:
# renomeando a coluna "situacao_candidado" para "perfil_compativel"
df_unido = df_unido.rename(columns={"situacao_candidado": "perfil_compativel"})
df_unido.head(5)

,codigo_condidato,codigo_vaga,perfil_compativel,titulo_vaga,cliente,tipo_contratacao,vaga_especifica_para_pcd,nivel profissional,nivel_academico,nivel_ingles,principais_atividades,competencia_tecnicas_e_comportamentais,cv_pt
0,25632,4530,sim,CONSULTOR CONTROL M,"Morris, Moran and Dodson",PJ/Autônomo,Não,Pleno,Ensino Superior Completo,Nenhum,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,\ndados pessoais\nestado civil: casado\nidade:...
1,25529,4530,sim,CONSULTOR CONTROL M,"Morris, Moran and Dodson",PJ/Autônomo,Não,Pleno,Ensino Superior Completo,Nenhum,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,"solteiro, 47 anos\n\nestrada meringuava, nº 17..."
3,25360,4531,sim,2021-2607395-PeopleSoft Application Engine-Dom...,Gonzalez and Sons,PJ/Autônomo,Não,Sênior,Ensino Médio Completo,Nenhum,Key skills required for the job are:\n\nPeople...,O recurso Peoplesoft tem como responsabilidade...,informações pessoais\n estado civil: casado\n...
8,26003,4534,não,2021-2605711-Microfocus QTP - UFT Automation T...,Barnes-Woods,PJ/Autônomo,Não,Sênior,Ensino Médio Completo,Técnico,Automação de teste (conhecimento do código)\n\...,Automação de teste (conhecimento do código)\n\...,solteira – 40 anos – brasileira\nitaquaquecetu...
11,21458,4534,sim,2021-2605711-Microfocus QTP - UFT Automation T...,Barnes-Woods,PJ/Autônomo,Não,Sênior,Ensino Médio Completo,Técnico,Automação de teste (conhecimento do código)\n\...,Automação de teste (conhecimento do código)\n\...,pexperiência\nfoursys\nsoftware tester\nabril ...


In [13]:
df_unido.to_csv("dados/df_final.csv", index=False)